In [1]:
'''This notebook is intended to demonstrate how select registration, segmentation, 
and image mathematical methods of ITKTubeTK can be combined to perform multi-channel 
brain extraction (aka. skull stripping for patient data containing multiple MRI sequences).

There are many other (probably more effective) brain extraction methods available as 
open-source software such as BET and BET2 in the FSL package (albeit such methods are only 
for single channel data). If you need to perform brain extraction for a large collection 
of scans that do not contain major pathologies, please use one of those packages. 
This notebook is meant to show off the capabilities of specific ITKTubeTK methods, 
not to demonstration how to "solve" brain extraction.'''

'This notebook is intended to demonstrate how select registration, segmentation, \nand image mathematical methods of ITKTubeTK can be combined to perform multi-channel \nbrain extraction (aka. skull stripping for patient data containing multiple MRI sequences).\n\nThere are many other (probably more effective) brain extraction methods available as \nopen-source software such as BET and BET2 in the FSL package (albeit such methods are only \nfor single channel data). If you need to perform brain extraction for a large collection \nof scans that do not contain major pathologies, please use one of those packages. \nThis notebook is meant to show off the capabilities of specific ITKTubeTK methods, \nnot to demonstration how to "solve" brain extraction.'

In [2]:
import os
import numpy as np

import itk
from itk import TubeTK as ttk

from itkwidgets import view

In [3]:
MRA_path = '/media/peirong/PR/IXI/IXI-MRA/IXI002-Guys-0828-MRA.nii.gz'
T1_path = '/media/peirong/PR/IXI/IXI-T1/IXI002-Guys-0828-T1.nii.gz'
T2_path = '/media/peirong/PR/IXI/IXI-T2/IXI002-Guys-0828-T2.nii.gz'

save_fld = '/media/peirong/PR/IXI/IXI-Vessel/level-0'

In [9]:

ImageType = itk.Image[itk.F, 3]
ReaderType = itk.ImageFileReader[ImageType]

reader1 = ReaderType.New(FileName=MRA_path)
reader1.Update()
im1 = reader1.GetOutput()

reader2 = ReaderType.New(FileName=T1_path)
reader2.Update()
im2 = reader2.GetOutput()

reader3 = ReaderType.New(FileName=T2_path)
reader3.Update()
im3 = reader3.GetOutput()

In [10]:
# Resample data into isotropic voxels and register T1 and T2 with MRA

res = ttk.ResampleImage.New(Input = im1) 
res.SetMakeHighResIso(True)
res.Update()
im1iso = res.GetOutput()

imReg = ttk.RegisterImages[ImageType].New()  # This is the standard protocol for within patient within visit registration
imReg.SetFixedImage(im1iso)
imReg.SetMovingImage(im2)
imReg.SetReportProgress(True)
imReg.SetExpectedOffsetMagnitude(40)
imReg.SetExpectedRotationMagnitude(0.01)
imReg.SetExpectedScaleMagnitude(0.01)
imReg.SetRigidMaxIterations(500)
imReg.SetRigidSamplingRatio(0.1)
imReg.SetRegistration("RIGID")
imReg.SetMetric("MATTES_MI_METRIC")
imReg.Update()
im2iso = imReg.GetFinalMovingImage("LINEAR_INTERPOLATION") # NOTE: args added

imReg = ttk.RegisterImages[ImageType].New()
imReg.SetFixedImage(im1iso)
imReg.SetMovingImage(im3)
imReg.SetReportProgress(True)
imReg.SetExpectedOffsetMagnitude(40)
imReg.SetExpectedRotationMagnitude(0.01)
imReg.SetExpectedScaleMagnitude(0.01)
imReg.SetRigidMaxIterations(500)
imReg.SetRigidSamplingRatio(0.1)
imReg.SetRegistration("RIGID")
imReg.SetMetric("MATTES_MI_METRIC")
imReg.Update()
im3iso = imReg.GetFinalMovingImage("LINEAR_INTERPOLATION") # NOTE: args added

In [11]:
CheckerboardImagesType = itk.CheckerBoardImageFilter[ImageType]
cb23 = CheckerboardImagesType.New(Input1=im2iso, Input2=im3iso)
cb23.Update()
im23 = ImageType.New()
im23 = cb23.GetOutput()
view(im23)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [12]:
N = 8
BaseLineFld = '/media/peirong/PR/TubeTK_Data'
readerList = ["003", "010", "026", "034", "045", "056", "063", "071"]

imBase = []
imBaseB = []
for i in range(0,N):
    name = os.path.join(BaseLineFld, "Normal"+readerList[i]+"-FLASH.mha")
    nameB = os.path.join(BaseLineFld, "Normal"+readerList[i]+"-FLASH-Brain.mha")
    reader = ReaderType.New(FileName=name)
    reader.Update()
    imBaseTmp = reader.GetOutput()
    reader = ReaderType.New(FileName=nameB)
    reader.Update()
    imBaseBTmp = reader.GetOutput()
    imBase.append(imBaseTmp)
    imBaseB.append(imBaseBTmp)

In [ ]:
imMath = ttk.ImageMath.New(Input=im2iso)
imMath.Blur(2)
im2isoBlur = imMath.GetOutput()

regB = []
regBB = []
print('Start')
for i in range(0,N):
    print(i)
    imMath.SetInput(imBase[i])
    imMath.Blur(2)
    imBaseBlur = imMath.GetOutput()
    regBTo1 = ttk.RegisterImages[ImageType].New(FixedImage=im2isoBlur, MovingImage=imBaseBlur)
    regBTo1.SetReportProgress(True)
    regBTo1.SetExpectedOffsetMagnitude(40)
    regBTo1.SetExpectedRotationMagnitude(0.01)
    regBTo1.SetExpectedScaleMagnitude(0.1)
    regBTo1.SetRigidMaxIterations(500)
    regBTo1.SetAffineMaxIterations(500)
    regBTo1.SetRigidSamplingRatio(0.1)
    regBTo1.SetAffineSamplingRatio(0.1)
    regBTo1.SetInitialMethodEnum("INIT_WITH_IMAGE_CENTERS")
    regBTo1.SetRegistration("PIPELINE_AFFINE")
    regBTo1.SetMetric("MATTES_MI_METRIC")
    #regBTo1.SetMetric("NORMALIZED_CORRELATION_METRIC") - Really slow!
    #regBTo1.SetMetric("MEAN_SQUARED_ERROR_METRIC")
    regBTo1.Update()
    img = regBTo1.ResampleImage("LINEAR", imBase[i])
    regB.append( img )
    img = regBTo1.ResampleImage("LINEAR", imBaseB[i])
    regBB.append( img )

In [15]:
regBBT = []
for i in range(0,N):
    print(i)
    imMath = ttk.ImageMath[ImageType,ImageType].New( Input=regBB[i] )
    imMath.Threshold(0,1,0,1)
    img = imMath.GetOutput()
    if i==0:
        imMathSum = ttk.ImageMath[ImageType,ImageType].New( img )
        imMathSum.AddImages( img, 1.0/N, 0 )
        sumBBT = imMathSum.GetOutput()
    else:
        imMathSum = ttk.ImageMath[ImageType,ImageType].New( sumBBT )
        imMathSum.AddImages( img, 1, 1.0/N )
        sumBBT = imMathSum.GetOutput()
        
view(sumBBT)

0
1
2
3
4
5
6
7


Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [16]:

insideMath = ttk.ImageMath[ImageType,ImageType].New( Input = sumBBT )
insideMath.Threshold(1,1,1,0)
insideMath.Erode(5,1,0)
brainInside = insideMath.GetOutput()

outsideMath = ttk.ImageMath[ImageType,ImageType].New( Input = sumBBT )
outsideMath.Threshold(0,0,1,0)
outsideMath.Erode(10,1,0)
brainOutsideAll = outsideMath.GetOutput()
outsideMath.Erode(20,1,0)
outsideMath.AddImages(brainOutsideAll, -1, 1)
brainOutside = outsideMath.GetOutput()

outsideMath.AddImages(brainInside,1,2)
brainCombinedMask = outsideMath.GetOutputUChar()

outsideMath.AddImages(im2iso, 512, 1)
brainCombinedMaskView = outsideMath.GetOutput()
view(brainCombinedMaskView)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [17]:
LabelMapType = itk.Image[itk.UC,3]

segmenter = ttk.SegmentConnectedComponentsUsingParzenPDFs[ImageType,LabelMapType].New()
segmenter.SetFeatureImage( im1iso )
segmenter.AddFeatureImage( im2iso )
segmenter.AddFeatureImage( im3iso )
segmenter.SetInputLabelMap( brainCombinedMask )
segmenter.SetObjectId( 2 )
segmenter.AddObjectId( 1 )
segmenter.SetVoidId( 0 )
segmenter.SetErodeDilateRadius( 5 )
segmenter.Update()
segmenter.ClassifyImages()
brainCombinedMaskClassified = segmenter.GetOutputLabelMap()

In [18]:
cast = itk.CastImageFilter[LabelMapType, ImageType].New()
cast.SetInput(brainCombinedMaskClassified)
cast.Update()
brainMaskF = cast.GetOutput()

brainMath = ttk.ImageMath[ImageType,ImageType].New(Input = brainMaskF)
brainMath.Threshold(2,2,1,0)
brainMath.Dilate(2,1,0)
brainMaskD = brainMath.GetOutput()
brainMath.SetInput( im1 )
brainMath.ReplaceValuesOutsideMaskRange( brainMaskD, 1, 1, 0)
brain = brainMath.GetOutput()

view(brain)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [20]:
itk.imwrite(brainMaskD, os.path.join(save_fld, "MRMask-Brain.mha"))

itk.imwrite(im1iso, os.path.join(save_fld, "MRA-Iso.mha"))
itk.imwrite(im2iso, os.path.join(save_fld, "MRT1-Iso.mha"))
itk.imwrite(im3iso, os.path.join(save_fld, "MRT2-Iso.mha"))

itk.imwrite(brain, os.path.join(save_fld, "MRA-Brain.mha"))

brainMath.SetInput(im2iso)
brainMath.ReplaceValuesOutsideMaskRange( brainMaskD, 1, 1, 0 )
itk.imwrite(brainMath.GetOutput(), os.path.join(save_fld, "MRT1-Brain.mha"))

brainMath.SetInput(im3iso)
brainMath.ReplaceValuesOutsideMaskRange( brainMaskD, 1, 1, 0 )
itk.imwrite(brainMath.GetOutput(), os.path.join(save_fld, "MRT2-Brain.mha"))